### Задание №1: Собственные функции


In [2]:
# 1. Функция создания словаря из двух списков
def lists_to_dict(keys, values):
    if len(keys) != len(values):
        raise ValueError("Длины списков не совпадают")
    return dict(zip(keys, values))

# 2. Функция группировки слов по длине
def group_by_length(words):
    groups = {}
    for word in words:
        length = len(word)
        if length not in groups:
            groups[length] = []
        groups[length].append(word)
    for key in groups:
        groups[key].sort()
    return groups

# 3. Функция преобразования числового списка
def transform_numbers(numbers):
    return [x**2 if x % 2 == 0 else x**3 for x in numbers]

keys = [1, 2, 3]
values = [4, 5, 6]
print("Словарь из списков:", lists_to_dict(keys, values))

words = ["apple", "banana", "cherry", "date", "fig"]
print("Группировка слов по длине:", group_by_length(words))

numbers = [1, 2, 3, 4, 5]
print("Преобразованный список чисел:", transform_numbers(numbers))

Словарь из списков: {1: 4, 2: 5, 3: 6}
Группировка слов по длине: {5: ['apple'], 6: ['banana', 'cherry'], 4: ['date'], 3: ['fig']}
Преобразованный список чисел: [1, 4, 27, 16, 125]


### Задание №2: Рекурсия


In [ ]:
# 1. Рекурсивное генерирование всех перестановок списка
def permute(lst):
    if len(lst) == 0:
        return []
    if len(lst) == 1:
        return [lst]
    result = []
    for i in range(len(lst)):
        m = lst[i]
        rem_lst = lst[:i] + lst[i+1:]
        for p in permute(rem_lst):
            result.append([m] + p)
    return result

# 2. Рекурсивное генерирование сочетаний заданной длины
def combinations(lst, k):
    if k == 0:
        return [[]]
    if not lst or k > len(lst):
        return []
    head = lst[0]
    tail = lst[1:]
    return [ [head] + c for c in combinations(tail, k-1) ] + combinations(tail, k)

# 3. Рекурсивное построение степенного множества
def power_set(lst):
    if not lst:
        return [[]]
    head = lst[0]
    tail_power_set = power_set(lst[1:])
    return tail_power_set + [ [head] + subset for subset in tail_power_set ]

print("Перестановки списка [1,2,3]:", permute([1, 2, 3]))
print("Сочетания из 2 элементов:", combinations([1, 2, 3], 2))
print("Степенное множество [1,2,3]:", power_set([1, 2, 3]))

Перестановки списка [1,2,3]: [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
Сочетания из 2 элементов: [[1, 2], [1, 3], [2, 3]]
Степенное множество [1,2,3]: [[], [3], [2], [2, 3], [1], [1, 3], [1, 2], [1, 2, 3]]


### Задание №3: Функции высших порядков


In [4]:
import time

# 1. Декоратор «time_it»
def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} выполнилась за {end - start:.6f} секунд")
        return result
    return wrapper

# 2. Декоратор мемоизации
def memoize(func):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrapper

# 3. Декоратор «retry»
def retry(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < n:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts == n:
                        raise e
        return wrapper
    return decorator

@time_it
def sum_large_list():
    return sum(range(1000000))

sum_large_list()

@memoize
def factorial(n):
    return 1 if n == 0 else n * factorial(n-1)

print("Факториал 5 с мемоизацией:", factorial(5))

@retry(3)
def risky_func():
    import random
    if random.random() < 0.5:
        raise ValueError("Случайная ошибка")
    return "Успех"

try:
    print(risky_func())
except Exception as e:
    print(f"Ошибка после повторов: {e}")

sum_large_list выполнилась за 0.038065 секунд
Факториал 5 с мемоизацией: 120
Успех


### Задание №4. Декораторы


In [5]:
from functools import wraps
def type_check(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        annotations = func.__annotations__
        
        # Проверка позиционных аргументов
        for i, (arg_name, arg_type) in enumerate(annotations.items()):
            if i >= len(args):
                break
            if not isinstance(args[i], arg_type):
                raise TypeError(f"Аргумент {arg_name} должен быть типа {arg_type}")
        
        # Проверка именованных аргументов
        for arg_name, arg_value in kwargs.items():
            if arg_name in annotations and not isinstance(arg_value, annotations[arg_name]):
                raise TypeError(f"Аргумент {arg_name} должен быть типа {annotations[arg_name]}")
        
        return func(*args, **kwargs)
    return wrapper

def cache_with_counter(func):
    cache = {}
    func.call_count = 0
    
    @wraps(func)
    def wrapper(*args):
        nonlocal cache
        func.call_count += 1
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    
    return wrapper

@type_check
def typed_func(a: int, b: str) -> str:
    return b * a

try:
    print(typed_func(3, "hello"))
except TypeError as e:
    print(f"Ошибка: {e}")

try:
    print(typed_func("three", "hello"))
except TypeError as e:
    print(f"Ошибка: {e}")
    
@cache_with_counter
def factorial(n):
    return 1 if n <= 1 else n * factorial(n-1)
print(factorial(5))
print(f"Количество вызовов: {factorial.call_count}")

hellohellohello
Ошибка: Аргумент a должен быть типа <class 'int'>
120
Количество вызовов: 0


### Задание №5. Основы функционального программирования


In [6]:
# 1. Группировка чисел по чётности
def group_by_parity(numbers):
    return {
        'even': sorted(filter(lambda x: x % 2 == 0, numbers)),
        'odd': sorted(filter(lambda x: x % 2 != 0, numbers))
    }

# 2. Генерация последовательности Фибоначчи
def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        fib_prev = fibonacci(n - 1)
        fib_prev.append(fib_prev[-1] + fib_prev[-2])
        return fib_prev

print("Группировка чисел по чётности:", group_by_parity([3, 1, 4, 1, 5, 9, 2, 6]))
print("Первые 10 чисел Фибоначчи:", fibonacci(10))

Группировка чисел по чётности: {'even': [2, 4, 6], 'odd': [1, 1, 3, 5, 9]}
Первые 10 чисел Фибоначчи: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
